In [2]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import requests

In [3]:
requests.get("http://localhost:11434").content

b'Ollama is running'

In [4]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [5]:
# constants
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
MODEL_LLAMA = 'llama3.2'

In [6]:
system_promt= """ 
                You are a technical explainer and problem-solver.
Your job is to answer the user’s technical question with a clear, accurate explanation that matches their skill level.

Guidelines:
- Start with a short direct answer.
- Then explain the concept step-by-step in plain language.
- Use a small example when helpful (code, command, or pseudo-code).
- Call out assumptions and edge cases.
- If the question is ambiguous, ask 1–2 clarifying questions, otherwise make reasonable assumptions and state them.
- Be honest about uncertainty; do not invent facts.
- Prefer practical guidance over theory unless the user asks for theory.
Output format:
1) Direct answer
2) Explanation
3) Example (if useful)
4) Common pitfalls / gotchas
5) Next steps (optional)

Respond in markdown with proper label and sub label without code blocks.

"""

In [7]:
# here is the question; type over this to ask something new
def get_question_user_prompt(question):
    question_user_prompt = """ Please explain what this code does and why:
                                   \n """
    question_user_prompt +=question 
    return question_user_prompt
  
#           

In [8]:
question=' yield from {book.get("author") for book in books if book.get("author")} '
get_question_user_prompt(question)

' Please explain what this code does and why:\n                                   \n  yield from {book.get("author") for book in books if book.get("author")} '

In [9]:
def stream_code_explainer_ollama(question):
    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_promt},
            {"role": "user", "content": get_question_user_prompt(question)}
          ],
        stream=True  # streanms one by one in chunks (parts)
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [10]:
# Get Llama 3.2 to answer
stream_code_explainer_ollama(question)

**Direct Answer**
The given code uses a generator expression to extract the authors of books from a list of dictionaries.


**Explanation**

This code is a mix of two advanced concepts: iterators and dictionary methods. Here's how it works:

*   The `yield from` expression is used to delegate sub-iteration for the resulting iterator.
*   The `.get("author") for book in books if book.get("author")` part is a filter. It filters out any dictionaries (`book`) that don't contain an "author" key.

**Example**

Suppose we have a list of dictionaries representing books, where each dictionary has an "author" key:

```python
books = [
    {"title": "Book One", "author": "Author1"},
    {"title": "Book Two", "author": "Author2"},
    {"title": "Another Book"}
]
```

The code would process the first two books, because they contain the "author" key:

```python
authors = yield from {book.get("author") for book in books if book.get("author")}
# authors -> ['Author1', 'Author2']
```

But it would skip the third, since its dictionary didn't contain an "author" key.


**Common Pitfalls / Gotchas**

*   If you are not careful, using `.get()` can result in keys being treated as attributes with non-dictionary values. In other words, if a book object is missing the author key and has `book.author`, things won't behave correctly.
*   Don’t forget to handle exceptions while using `.get()`. There will always be potential edge cases.

**Next Steps**
Consider replacing `'Book'` in code with an array or list containing strings to further process these filtered book names.

In [11]:
question=""""week1 EXERCISE.ipynb"links = fetch_website_links(url)
           user_prompt += "\n".join(links) """
stream_code_explainer_ollama(question)

**Direct Answer**
This code downloads a list of links from a webpage, appends the links to a string, and saves the response for later use.

**Explanation**

This code snippet is part of a larger script that interacts with a website. Here's a step-by-step breakdown:

1. `fetch_website_links(url)`: This function fetches (downloads) a list of links from a given URL on the internet.
2. `week1 EXERCISE.ipynb links = ...`: The `links` variable contains the returned result of `fetch_website_links(url).
3. `" ".join(links)`: This expression concatenates all the links in the `links` list into a single string, separated by spaces.
4. `user_prompt += ...`: The resulting string is appended to a larger string called `user_prompt`.

**Example**

```python
def fetch_website_links(url):
    # Replace this with your actual website URL and scraping logic
    return ["https://www.example.com", "https://www.google.com"]

links = fetch_website_links("https://www.example.com")
new_links_string = " ".join(links)
user_prompt += new_links_string
```

**Common Pitfalls / Gotchas**

* Make sure you're respecting the website's `robots.txt` file and scraping policies to avoid getting banned or reported.
* Be cautious when dealing with dynamically generated content, as it might require additional logging, JavaScript execution, or user interaction.

Next Steps:

Since you've asked for an explanation, if your question has more context (like what "fetch_website_links" function does or what kind of data is stored in `user_prompt`), I'd love to ask a few clarifying questions:

* Can you explain where this script runs (e.g., Jupyter Notebook, command-line interface)?
* What is the purpose of `user_prompt` in this context?
* Does the code handle errors or edge cases (e.g., non-HTTP responses or server-side scraping restrictions)?

### Adding UI using gradio

In [22]:
import gradio as gr
import re

In [25]:
def looks_like_code(s: str) -> bool:
    s = s.strip()
    if not s:
        return False

    patterns = [
        r"```",                          # fenced code blocks
        r"^\s{4,}\S",                    # indented lines (4+ spaces)
        r"\b(def|class|import|from|if|elif|else|for|while|try|except|with|return|print)\b",
        r"[(){}\[\];]",                  # code punctuation
        r"==|!=|<=|>=|:=|\+=|-=|\*=|/=|//=|\*\*",
        r":\s*$",                        # line ending with colon (Python blocks)
        r"#",                            # comment
    ]

    score = sum(bool(re.search(p, s, re.MULTILINE)) for p in patterns)
    if score >= 2:
        likely_code=True
    else:
        likely_code=False

    return likely_code



In [ ]:
def generate_code_explainer_ollama(message,history):
    history=[{"role":h['role'],"content":h['content']}for h in history]
    if looks_like_code(message)==True:
        relative_system_promt=system_promt
        relative_user_prompt=get_question_user_prompt(message)
    else:
        relative_system_promt='You are an polite helpful assistant'
        relative_user_prompt=message
        
    
    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[{"role": "system", "content": relative_system_promt}] +history+
          [{"role": "user", "content": relative_user_prompt}],
        stream=True  # streanms one by one in chunks (parts)
    )    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [27]:
chat=gr.ChatInterface(fn=generate_code_explainer_ollama,title='Code Explainer')
chat.launch(share=True)

* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://f57e517315f0b2f1b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
